In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
from pprint import pprint
import pylab

In [11]:
%matplotlib inline

In [12]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [13]:
exps = ['bert-base', 'bert-large', 'roberta-base', 'roberta-large']

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

def get_average_val(lines):
    reported = []
    for line in lines:
        if len(line.split('\t')) != 2:
            print('\t??', line)
        val = float(line.split('\t')[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out

In [14]:
all_tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI","RTE"]

result_folder = "mt_dnn_exp_results"
name_mapping = {
    "bert":"BERT",
    "roberta":"roBERTa",
    "xlnet":"XLNet"
}

results = {}

for task in all_tasks:
    print("\n")
    results[task] = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        task_results = {
            'base':{
                'all':[]
            },
            'large':{
                'all':[]
            }
        }
        
        for exp in exps:

            # base metrics
            f_name = f"../../{result_folder}/{exp}/{task}/base-{metric}.txt" 
#             print(f_name)
            try:
                f=open(f_name, "r")
                try:
                    lines = f.read().splitlines()

                    if "base" in exp:
                        task_results['base']['all'].append(get_average_val(lines))
                    elif "large" in exp:
                        task_results['large']['all'].append(get_average_val(lines))

                except e:
                    print("failed to process 1", f_name, e)
                    if "base" in exp:
                        task_results['base']['all'].append(0)
                    elif "large" in exp:
                        task_results['large']['all'].append(0)
                        
            except:
                print("failed to open 1", f_name)
                if "base" in exp:
                    task_results['base']['all'].append(0)
                elif "large" in exp:
                    task_results['large']['all'].append(0)
                    

                    
            # no layer metrics

            fine_tuning_metrics = []
            f_name=f"../../{result_folder}/{exp}/{task}/no_layer-{metric}.txt"
#             print(f_name)
            
            try:
                f=open(f_name, "r")
                
                try:

                    lines = f.read().splitlines()
                    fine_tuning_metrics.append(get_average_val(lines))
                
                except e:
                    print("failed to process 2 ", f_name, e)
                    fine_tuning_metrics.append(0)

            except:
                print("failed to open 2", f_name)
                fine_tuning_metrics.append(0)

            # fine-tuned metrics

            if "base" in exp:
                num_layers = 12
            elif "large" in exp:
                num_layers = 24
            
            log_file_prefix=''
            for i in reversed(range(int(num_layers/2), num_layers)):
                log_file_prefix += str(i)
                f_name = f"../../{result_folder}/{exp}/{task}/{log_file_prefix}-{metric}.txt"
#                 print(f_name)
                try:
                    f=open(f_name, "r")
                    try:                        
                        lines = f.read().splitlines()
                        fine_tuning_metrics.append(get_average_val(lines))
                        
                    except e:
                        
                        
                        print("failed to process 3", f_name, e)
                        print(lines)
                        fine_tuning_metrics.append(0)
                except:
                    print("failed to open 3", f_name)
                    fine_tuning_metrics.append(0)
                
                log_file_prefix +='_'

#             task_results[exp] = list(reversed(fine_tuning_metrics))

            model_name = name_mapping[exp.split('-')[0]]
            
            if "base" in exp:
                task_results['base'][model_name] = list(reversed(fine_tuning_metrics))
            elif "large" in exp:
                task_results['large'][model_name] = list(reversed(fine_tuning_metrics))
                
        results[task][metric] = task_results




failed to open 1 ../../mt_dnn_exp_results/roberta-large/CoLA/base-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17_16-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17_16_15-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17_16_15_14-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17_16_15_14_13-mcc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/CoLA/23_22_21_20_19_18_17_16_15_14_13_12-mcc.txt


failed to open 1 ../../mt_dnn_exp_results/roberta-large/SST-2/base-acc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/SST-2/23_22_21_20_19_18-acc.txt
failed to open 3 ../../mt_dnn_exp_results/roberta-large/SST-2/23_22_21_20_19_18_17-acc.

In [15]:
color_mapping = {
    "bert":'green',
    "roberta":'red',
    "xlnet":'blue',
    "BERT":'green',
    "roBERTa":'red',
    "XLNet":'blue'
}

y_label_mapping = {
    "mcc":'MCC',
    "f1":r'$F_1$',
    "acc":'Acc.',
    "spearmanr": r'$\rho$'
}

models = ["BERT", "roBERTa", "XLNet"]

legends = None

def draw_graph(model_size, task, y_label, collected):
    fig = plt.figure(figsize=(6,6))
    
    # x axis
    x_axis = []
    
    if "base" in model_size:
        num_layers = 12
    elif "large" in model_size:
        num_layers = 24

    for i in range(int(num_layers/2), num_layers):
        x_axis.append(str(i))
#     x_axis.append(f"{num_layers} (all)")
    x_axis.append("all")
    
    x_axis.reverse()
                
    legends = []

    base_perf = []
    
    for ind, model in enumerate(models):
        if collected['all'][ind] != 0:
            base_perf.append(collected['all'][ind])
    
    temp_base = np.mean(base_perf)
    
#     # baseline
    for ind, model in enumerate(models):
#         handle = plt.axhline(y=collected['all'][ind], linestyle='--', label=model, c=color_mapping[model])
#         legends.append(handle)

        plt.axhline(y=0, linestyle='--', c='grey', label='_nolegend_')
    
        temp = collected[model]
        temp.reverse()
        
        if collected['all'][ind] == 0:
            relative = temp - temp_base
        else:
            relative = temp - collected['all'][ind]
    

    
        handle = plt.plot(x_axis, relative, linestyle='-', label=model, c=color_mapping[model])
        legends.append(handle)
        
    # y-lim
    if task == "SST-2" or task == "STS-B" or task == "MRPC":
        plt.ylim(-0.2, 0.049)
    else:
        plt.ylim(-0.6, 0.14)
        
    plt.rcParams.update({'font.size': 30})

    # legend
    
#     legend = plt.legend(legends[::2],     # The line objects
#        labels=["BERT", "RoBERTa", "XLNet"], 
#        loc=8,  # Position of legend
#        ncol=3,
#        bbox_to_anchor=(0.5, -0.5)
#        )

#     def export_legend(legend, filename=f'images/combined/legend.pdf', expand=[-5,-5,5,5]):
#         fig  = legend.figure
#         fig.canvas.draw()
#         bbox  = legend.get_window_extent()
#         bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
#         bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
#         fig.savefig(filename, dpi="figure", bbox_inches=bbox)

#     export_legend(legend)

    
    if "base" in model_size:
        plt.xticks(x_axis, fontsize=25)
    elif "large" in model_size:
        plt.xticks(x_axis[::2], fontsize=25)

    plt.yticks(fontsize=25)
    
#     plt.xlabel("Number of frozen layers", fontsize=30)
    plt.ylabel(y_label_mapping[y_label], fontsize=30)
    
    plt.title(f'{task}-{model_size}', fontsize=30)
    plt.savefig(f'images/combined/{task}-{model_size}-diff-nl.pdf', format='pdf', bbox_inches='tight')
#     plt.savefig(f'images/combined/{task}-{model_size}-diff.pdf', format='pdf', bbox_inches='tight')
    plt.show()

In [16]:
model_size = 'base'
tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI","RTE"]
tasks = ["CoLA","MRPC","STS-B","RTE", "SST-2"]

for task in tasks:
    task_metrics = metrics[task]
    for metric in task_metrics:
        draw_graph(model_size, task, metric, results[task][metric][model_size])

IndexError: list index out of range

<Figure size 432x432 with 0 Axes>

In [ ]:
model_size = 'large'
tasks = ["CoLA","MRPC","STS-B","RTE", "SST-2"]

for task in tasks:
    task_metrics = metrics[task]
    for metric in task_metrics:
        draw_graph(model_size, task, metric, results[task][metric][model_size])

In [ ]:
# plt.figure(figsize=(10,6))
# plt.title('TEMP IMAGE', fontsize=23)
# plt.savefig(f'images/combined/temp.png', format='png', bbox_inches='tight')